In [1]:
import pandas as pd
import numpy as np
import operator

Importo la base de datos de los pacientes, recogida en SPSS y pasada posteriormente a documento csv. Para su analisis

In [2]:
datos=pd.read_csv("Analisis_ano.csv",sep=";")

In [3]:
datos.head()

,Médico,Tipo_cápsula,Fecha_procedimiento,ID,Sexo,Nacimiento,Edad,Cintura,IMC,Tiempo_estudio,...,Patency,No_aporta_previo,Eco_abdominal,Rx_abdomen,Entero_TAC,Entero_RM,Tránsito_intes,Endoscopia_previa,Intro_cápsula,Complicaciones
0,1,1,5/14/2012,1,1,6/24/1925,86,,,,...,0,1,0,0,0,0,0,0,0,0
1,1,1,9/5/2012,2,1,1/12/1953,59,,,,...,0,1,0,0,0,0,0,0,0,0
2,1,1,10/17/2012,3,1,2/21/1951,61,,,,...,0,1,0,0,0,0,0,0,0,0
3,1,1,12/14/2013,4,1,10/1/1950,63,,,,...,0,1,0,0,0,0,0,0,0,0
4,1,1,10/25/2013,5,0,7/4/1935,78,,,,...,0,1,0,0,0,0,0,2,0,0


In [4]:
datos.shape

(251, 150)

In [5]:
datos.dtypes[:]

Médico                  int64
Tipo_cápsula            int64
Fecha_procedimiento    object
ID                      int64
Sexo                    int64
                        ...  
Entero_RM               int64
Tránsito_intes          int64
Endoscopia_previa      object
Intro_cápsula           int64
Complicaciones         object
Length: 150, dtype: object

In [6]:
columnas=datos.columns

In [7]:
incompletas=[]
for e in columnas:
    if datos[e].isnull().sum()>=1:
        incompletas.append(e)
print(incompletas)

[]


Como se ve no hay ninguna variable, con valores nulos en toda la base de datos

In [8]:
columnas=datos.columns
columnas.tolist()

['Médico',
 'Tipo_cápsula',
 'Fecha_procedimiento',
 'ID',
 'Sexo',
 'Nacimiento',
 'Edad',
 'Cintura',
 'IMC',
 'Tiempo_estudio',
 'T_estómago',
 'T_ID_total',
 'T_colon_total',
 'T_expulsión',
 'Expulsa_cápsula',
 'HDOO_oculta',
 'HDOO_mani',
 'Crohn',
 'Alt_tránsito',
 'Anemia',
 'Dolor_abd',
 'Estreñimiento',
 'Diarrea',
 'Vómitos',
 'Pérdida_peso',
 'EII',
 'Colono_imcp',
 'Cribado',
 'Revisión',
 'Otros_indica',
 'Hernia_hiato_esof',
 'RGE_esof',
 'Trast_tránsito_esof',
 'Erosiones_esof',
 'Barret_esof',
 'Esof_eosinofilica_esof',
 'Estenosis_extr_esof',
 'No_evaluable_esof',
 'No_informado_esof',
 'Normal_esof',
 'Normal_estom',
 'No_informado_estom',
 'No_evaluable_estom',
 'Hernia_hiato_estom',
 'Trast_motilidad_estom',
 'Erosiones_estom',
 'Gastritis_estom',
 'Pólipo_estom',
 'Otros_estom',
 'No_informado_del',
 'No_evaluable_del',
 'Normal_del',
 'Hz_yeyuno',
 'Yeyunitis',
 'Erosiones_yey',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Hz_íleon',
 'Ileitis

In [9]:
#Borro la variable Dianoscitos porque sería un hallazgo compatible con la normalidad.
datos.drop(columns=['Dianocitos'],axis=0, inplace=True)

Uno en una única variable las hernias de hiato de esófago y estomago. Al ser el mismo diagnostico, simplemente puesto
en esófago o estomago según el momento de la cápsula en el que se diagnosticaba. Quedando unificada en la variable "Hernia_hiato_total"

In [10]:
def hernia(fila):
    a='Hernia_hiato_esof'
    b='Hernia_hiato_estom'
    if fila[a]==fila[b]:
        return fila[a]
    elif fila[a]<fila[b]:
        return fila[b]
    else:
        return fila[a]

In [11]:
datos["Hernia_hiato_total"]=datos.apply(lambda fila: hernia(fila),axis=1)

Compruebo que la reoorganización se ha realizado correctamente

In [12]:
comp=datos[['Hernia_hiato_esof','Hernia_hiato_estom',"Hernia_hiato_total"]].sort_values(by="Hernia_hiato_total"),
ascending=True
comp

(     Hernia_hiato_esof  Hernia_hiato_estom  Hernia_hiato_total
 0                    0                   0                   0
 150                  0                   0                   0
 151                  0                   0                   0
 152                  0                   0                   0
 153                  0                   0                   0
 ..                 ...                 ...                 ...
 84                   0                   1                   1
 212                  0                   1                   1
 210                  1                   0                   1
 188                  0                   1                   1
 125                  1                   0                   1
 
 [251 rows x 3 columns],)

Tras asegurarme, elimino las dos variables sobrantes

In [13]:
datos.drop(columns=['Hernia_hiato_esof','Hernia_hiato_estom'],axis=0, inplace=True)

# 1) Porcentaje de pacientes que vinieron con síntomas de dolor abdominal y tuvieron hallazgos en la cápsula. Diferenciando entre el centro en el que fue realizada la prueba.

Para ello, agrupo primero todas las varibles que representan hallazgos en la cápsula endoscopica. Los agrupo en dos 
tipos:
- Hallazgos que permiten tratamiento sin la necesidad de pruebas complementarias
- Hallazgos que harían necesaria la realización de pruebas complementaris para el diagnostico completo.

In [14]:
hz_Tta=["Hernia_hiato_total",
 'RGE_esof',
 'Erosiones_esof',
 'Erosiones_estom',
 'Pólipo_estom',
 'Telangiectasias_yey',
 'Linfagiectasias_yey',
 'Pólipo_yey',
 'Diverticulo_íleon',
'Erosiones_del',
 'Diverticulo_del',
 'Enteritis_del',
 'Pólipo_del',
 'Otros_del',
 'Hemorroides',
 'Diverticulo_col',
 'Diverticulo_C_drch',
 'Diverticulo_sig',
 'Diverticulo_C_izq',
 'Sigmoiditis',
 'Proctitis',
 'Angiodisplasia_col',
 'Pólipo_col',
 'Pólipo_C_drch',
 'Pólipo_C_asce',
 'Pólipo_C_trans',
 'Pólipo_C_izq',
 'Pólipo_C_desc',
 'Pólipo_ciego',
 'Pólipo_sig',
 'Pólipo_recto',
 'Melanosis',
 'Otras_col']

In [15]:
hz_prueba=['Trast_tránsito_esof',  'Barret_esof',  'Esof_eosinofilica_esof',  'Estenosis_extr_esof',  
           'Trast_motilidad_estom','Gastritis_estom',  'Yeyunitis',  'Erosiones_yey',  'Ileitis', 
           'Erosiones_íleon',  'Pólipo_íleon',  'Erosiones_col', 'Otros_estom',]

En cuanto a las variables otros hallazgos de estómago, delgado y colon decido incluirlas en hz_Tta porque en su
mayoria son de este tipo, aunque según resultados posteriores decidire si sacarlas, en base a posibles alteraciones 
en el analisis final.

In [16]:
cidma_dolor = datos[(datos['Médico']==0) & 
                (datos['Dolor_abd']==1)]

In [17]:
#Número de pacientes con dolor abdominal del centro CIDMA
cidma_dolor.shape 

(53, 148)

In [18]:
cidma_dolor["Hz_Tta"] = cidma_dolor[hz_Tta].sum(axis=1)

/usr/local/Cellar/ipython/7.7.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
#Número de pacientes con hallazgos para futuras terapías
len(cidma_dolor[cidma_dolor["Hz_Tta"]>=1])

39

In [20]:
cidma_dolor["Hz_Prueba"] = cidma_dolor[hz_prueba].sum(axis=1) 

/usr/local/Cellar/ipython/7.7.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
#Número de pacientes con hallazgos para futuros estudios
len(cidma_dolor[cidma_dolor["Hz_Prueba"]>=1]) 

33

In [22]:
#Número de pacientes con hallazgos compatibles con la normalidad
pct_normal=cidma_dolor[(cidma_dolor["Hz_Tta"]==0)&
(cidma_dolor["Hz_Prueba"]==0)]                     
len(pct_normal)

6

In [23]:
no_evaluable=['No_evaluable_esof','No_evaluable_estom','No_evaluable_del','No_evaluable_col']

In [24]:
#Número de pacientes con zonas del tracto GI sin posible evaluación
no_eva=[]
for e in no_evaluable:
    no_eva=cidma_dolor[cidma_dolor[e].isin([1])] 
len(no_eva)

11

In [25]:
for e in no_eva.index:
    for i in pct_normal.index:
        if e==i:
            print(e)

154


In [26]:
no_eva[no_evaluable].loc[154]

No_evaluable_esof     0
No_evaluable_estom    0
No_evaluable_del      0
No_evaluable_col      1
Name: 154, dtype: int64

El paciente 154 sería el único paciente sin hallazgos patológicos y con alguna zona del tracto digestivo sin poder evaluar (Colon). En cuanto al resto de pacientes con zonas no evaluables, destacar la zona colónica como la principal área no evaluada.

Ahora veré cuales son los pricipales hallazgos de dichos pacientes.
En primer lugar los hallazgos para tratamiento

In [27]:
dic={}
for e in hz_Tta:
    b=cidma_dolor[e].value_counts()
    dic[e]=b
    

In [28]:
cidma_dolor["Hernia_hiato_total"].value_counts()

0    39
1    14
Name: Hernia_hiato_total, dtype: int64

In [32]:
#Creo una serie con los valores de esta variable porque no introduje los 0 a la hora de creearla y da problemas
#al tratarla como un diccionario
dic["Hernia_hiato_total"]=pd.Series([29,14],index=[0,1]) 


In [30]:
hz_Tta_frec={}
for e in hz_Tta:
    if len(dic[e])>1:
        hz_Tta_frec[e]=dic[e][1]

In [31]:
sorted(hz_Tta_frec.items(), key=operator.itemgetter(1),reverse=True)

[('Diverticulo_col', 16),
 ('Hernia_hiato_total', 14),
 ('Hemorroides', 14),
 ('RGE_esof', 9),
 ('Otras_col', 8),
 ('Diverticulo_C_izq', 7),
 ('Diverticulo_sig', 6),
 ('Erosiones_estom', 5),
 ('Pólipo_estom', 5),
 ('Pólipo_col', 5),
 ('Otros_del', 4),
 ('Diverticulo_C_drch', 4),
 ('Linfagiectasias_yey', 3),
 ('Erosiones_del', 3),
 ('Enteritis_del', 3),
 ('Melanosis', 3),
 ('Angiodisplasia_col', 2),
 ('Erosiones_esof', 1),
 ('Diverticulo_íleon', 1),
 ('Pólipo_C_drch', 1),
 ('Pólipo_C_izq', 1),
 ('Pólipo_ciego', 1),
 ('Pólipo_sig', 1),
 ('Pólipo_recto', 1)]

Como se observa los tres hallazgos principales son:
- Diverticulos en colon
- Hernia de hiato
- Hemorroides

En cuanto a los hallazgos para completar estudio

In [33]:
dic={}
for e in hz_prueba:
    b=cidma_dolor[e].value_counts()
    dic[e]=b
    
hz_prueba_frec={}
for e in hz_prueba:
    if len(dic[e])>1:
        hz_prueba_frec[e]=dic[e][1]

In [34]:
sorted(hz_prueba_frec.items(), key=operator.itemgetter(1),reverse=True)

[('Gastritis_estom', 15),
 ('Yeyunitis', 5),
 ('Trast_tránsito_esof', 3),
 ('Erosiones_col', 3),
 ('Otros_estom', 3),
 ('Trast_motilidad_estom', 2),
 ('Erosiones_yey', 2),
 ('Ileitis', 2),
 ('Erosiones_íleon', 2),
 ('Barret_esof', 1),
 ('Esof_eosinofilica_esof', 1),
 ('Pólipo_íleon', 1)]

Principales hallazgos que necesitaran nuevas pruebas, destacan:
- Gastritis (Este triplica la frecuencia del segundo)
- Yeyunitis
- Trastornos del tránsito esofágico

Ahora analizaré las mismas variables en el hospital Italiano

In [35]:
uruguay_dolor = datos[(datos['Médico']==1) & 
                (datos['Dolor_abd']==1)]

In [36]:
#Número de pacientes con dolor abdominal en el hospital Italiano
len(uruguay_dolor["Médico"])

8

In [43]:
uruguay_dolor[hz_Tta].iloc[:,:20]

,Hernia_hiato_total,RGE_esof,Erosiones_esof,Erosiones_estom,Pólipo_estom,Telangiectasias_yey,Linfagiectasias_yey,Pólipo_yey,Diverticulo_íleon,Erosiones_del,Diverticulo_del,Enteritis_del,Pólipo_del,Otros_del,Hemorroides,Diverticulo_col,Diverticulo_C_drch,Diverticulo_sig,Diverticulo_C_izq,Sigmoiditis
45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
65,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
152,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
164,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
173,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [44]:
uruguay_dolor["Hz_Tta"] = uruguay_dolor[hz_Tta].sum(axis=1)

/usr/local/Cellar/ipython/7.7.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
#Número de pacientes con hallazgos para futuras terapías
len(uruguay_dolor[uruguay_dolor["Hz_Tta"]>=1])

5

In [48]:
uruguay_dolor["Hz_Prueba"] = uruguay_dolor[hz_prueba].sum(axis=1)

/usr/local/Cellar/ipython/7.7.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
#Número de pacientes con hallazgos para estudios complementarios
len(uruguay_dolor[uruguay_dolor["Hz_Prueba"]>=1])

3

In [50]:
#Número de pacientes con hallazgos compatibles con la normalidad
pct_normal_uru=uruguay_dolor[(uruguay_dolor["Hz_Tta"]==0)&
(uruguay_dolor["Hz_Prueba"]==0)]                    
len(pct_normal_uru)

3

In [80]:
uruguay_dolor[no_evaluable] #Número de pacientes con zonas del tracto GI sin posible evaluación (1)

,No_evaluable_esof,No_evaluable_estom,No_evaluable_del,No_evaluable_col
45,0,0,0,0
65,1,1,0,0
66,0,0,0,0
99,0,0,0,0
152,0,0,0,0
164,0,0,0,0
173,0,0,0,0
250,0,0,0,0


In [115]:
pct_normal_uru.index 

Int64Index([65, 164, 173], dtype='int64')

Como se ve uno de los pacientes (65) de sin hallazgos tiene el esófago y el estómago sin evaluar.

In [103]:
uruguay_dolor[hz_Tta].sum(axis=0)

Hernia_hiato_total     0.0
RGE_esof               0.0
Erosiones_esof         0.0
Erosiones_estom        0.0
Pólipo_estom           0.0
Telangiectasias_yey    0.0
Linfagiectasias_yey    0.0
Pólipo_yey             0.0
Diverticulo_íleon      0.0
Erosiones_del          0.0
Diverticulo_del        0.0
Enteritis_del          0.0
Pólipo_del             0.0
Otros_del              0.0
Hemorroides            0.0
Diverticulo_col        1.0
Diverticulo_C_drch     0.0
Diverticulo_sig        1.0
Diverticulo_C_izq      0.0
Sigmoiditis            2.0
Proctitis              0.0
Angiodisplasia_col     0.0
Pólipo_col             4.0
Pólipo_C_drch          1.0
Pólipo_C_asce          1.0
Pólipo_C_trans         0.0
Pólipo_C_izq           0.0
Pólipo_C_desc          0.0
Pólipo_ciego           0.0
Pólipo_sig             1.0
Pólipo_recto           1.0
Melanosis              0.0
Otras_col              1.0
dtype: float64

In [116]:
#Hallazgos para posterior estuido complementario 
uruguay_dolor[uruguay_dolor["Hz_Prueba"]>=1][hz_prueba]

,Trast_tránsito_esof,Barret_esof,Esof_eosinofilica_esof,Estenosis_extr_esof,Trast_motilidad_estom,Gastritis_estom,Yeyunitis,Erosiones_yey,Ileitis,Erosiones_íleon,Pólipo_íleon,Erosiones_col,Otros_estom
45,0,0,0,0,0,0,0,1,0,0,0,0,0
66,0,0,0,0,0,0,0,0,0,0,0,1,0
99,0,0,0,0,0,0,0,0,1,0,0,0,0


Hallazgos más frecuentes:
- Hz Tratamiento: Pólipos en colon (4), sigmoiditis, resto todos con 1.
- Hz Pruebas: Erosiones (Yeyuno, colon), ileitis.

In [154]:
pregunta_1=pd.DataFrame({"Pacientes":[53,8],"Normal":[[6,str(round((6*100/53),2))+" %"],[3,str(round((3*100/8),2))+" %"]], 
                         "No evaluable":[[11,str(round((11*100/53),2))+" %"],[1,str(round((1*100/8),2))+" %"]],
                       "Hz Pruebas":[[33,str(round((33*100/53),2))+" %"],[3,str(round((3*100/8),2))+" %"]],
                         "Hz Tratamiento":[[39,str(round((39*100/53),2))+" %"],[5,str(round((5*100/8),2))+" %"]]}
                        ,index=["Cidma","Uruguay"])
display(pregunta_1)


,Pacientes,Normal,No evaluable,Hz Pruebas,Hz Tratamiento
Cidma,53,"[6, 11.32 %]","[11, 20.75 %]","[33, 62.26 %]","[39, 73.58 %]"
Uruguay,8,"[3, 37.5 %]","[1, 12.5 %]","[3, 37.5 %]","[5, 62.5 %]"


# <font color='blue'>Número de pacientes y porcentajes</font>

Se observa que un 88,68% de los pacientes de CIDMA a los que se le hizo la cápsula por síntomas de dolor abdominal presentaron hallazgos de algún tipo. Destacando el 73,58% que recibieron un diagnostico definitivo, sin necesidad de pruebas complementarias. Sin embargo ese porcentaje cae al 62,5% en los pacientes del hospital Italiano, pero que aún así permite observar una capacidad de deteción de hallazgos de mas del 60%.

# 2 ) Porcentaje de pacientes que vinieron con hemorragia digestiva oculta en sangres y tuvieron hallazgos en la cápsula. Diferenciando entre el centro en el que fue realizada la prueba.


In [163]:
indicacion=['HDOO_oculta',
 'HDOO_mani',
 'Crohn',
 'Alt_tránsito',
 'Anemia',
 'Dolor_abd',
 'Estreñimiento',
 'Diarrea',
 'Vómitos',
 'Pérdida_peso',
 'EII',
 'Colono_imcp',
 'Cribado',
 'Revisión',
 'Otros_indica']

In [204]:
#Evaluó las otras indicaciones para analizar cuales son las más frecuentes
datos[indicacion].sum(axis=0)

HDOO_oculta      23
HDOO_mani        15
Crohn             4
Alt_tránsito      3
Anemia           25
Dolor_abd        61
Estreñimiento     6
Diarrea          35
Vómitos          10
Pérdida_peso      8
EII               5
Colono_imcp      10
Cribado          58
Revisión         18
Otros_indica     19
dtype: int64